## Creating a DataFrame

In my personal [cc_scraping.ipynb](Documents/Coding/Notebooks/tutorials/cc_scraping.ipynb) demo, I extracted data already from a pretty friendly F1 table.

What if I didn't have that? 

In this self-assigned exercise I will extract simple data from all the matches played in the EURO 2020 from the [Terrikon](https://terrikon.com/en/euro-2020) website. We will form a dataframe from the following info
1. Date
2. Stage of Competition
3. hTeam
4. aTeam
5. Score
6. Penalty Score

All using Requests, BeautifulSoup, then Pandas

In [1]:
import requests
from bs4 import BeautifulSoup

euro_url = "https://terrikon.com/en/euro-2020"
e_page = requests.get(euro_url)
e_html = e_page.text
e_soup = BeautifulSoup(e_html)

e_tables = e_soup.find("div", class_="maincol")
e_headers = e_tables.find_all("h2")
e_comps = ["".join(header.text.split("Euro-2020."))[1:] for header in e_headers]

##### Sections HTML by Matchtype
*i.e Final, Semi-Finals . . . Group F*

In [2]:
e_sections = []
for cnt, header in enumerate(e_headers):
    tracker = header.next_sibling
    cur_html = ""
    next_header = False
    try:
        next_header = e_headers[cnt +1]
    except:
        while tracker: 
            cur_html = cur_html + str(tracker)
            tracker = tracker.next_sibling
        if cur_html:
            e_sections.append(BeautifulSoup(cur_html).div)
        continue
    while next_header and tracker != next_header and tracker:
            cur_html = cur_html + str(tracker)
            tracker = tracker.next_sibling
    if cur_html:
            e_sections.append(BeautifulSoup(cur_html).div)

##### Attaches the relevant match type to the data
*i.e All Group A-F marked as Group Stage matches*



In [3]:
import pandas as pd
import io

tester = e_sections[0]
table = tester.select("table.gameresult")[0].prettify()
table = pd.read_html(io.StringIO(table))[0]

**To Note Above**

Pandas works best with pure html, in a fashion like so

`page = request.get(page_url) `<br>
`df = pd.read_html(io.StringIO(page.text), match="table title")`

Above, s_tester was formatted as beautifulSoup data, we had to prettify to get it back to original html text to parse through pandas 

So, this was an extra hurdle we made for ourselves. Now we know. However, our method above in code cell 71 was a great exercise to partition sections of the html if it wasn't handed to us like tables.

In [4]:
t_all_matches = []

In [5]:
for i, section in enumerate(e_sections): 
    s_tester = e_sections[i]
    s_table = s_tester.select("table.gameresult")[0].prettify()
    s_table = pd.read_html(io.StringIO(s_table))[0]
    s_table = s_table.drop(columns=[0])
    
    s_filler = [pd.NA for row in s_table.iterrows()]
    mType = e_comps[i]
    if "Group" in mType:
        mType = "Group Stage"
    s_matchtype = [mType for row in s_table.iterrows()]
    
    s_table = s_table.rename(columns={1:"hTeam", 2:"hGoals", 3:"aTeam", 4:"aGoals", 5:"Match Date"})
    s_table.insert(2, "hPenalties", s_filler, True)
    s_table.insert(5, "aPenalties", s_filler, True)
    s_table.insert(6, "Match Type", s_matchtype, True)
    
    for index, row in s_table.iterrows(): 
        scoring = row["hGoals"]
        hG = scoring[0]
        hP = pd.NA
        aG = scoring[2]
        aP = pd.NA
        if len(scoring) > 3: # penalties for match
            hP = scoring[6]
            aP = scoring[8]
            s_table.loc[index, "hPenalties"] = int(hP)
            s_table.loc[index, "aPenalties"] = int(aP)
    
        s_table.loc[index, "hGoals"] = int(hG)
        s_table.loc[index, "aGoals"] = int(aG)
    
    s_table[['hGoals', 'aGoals']] = s_table[['hGoals', 'aGoals']].astype('int')
    #s_table.dtypes
    t_all_matches.append(s_table)

In [6]:
t_matches_df = pd.concat(t_all_matches)
t_matches_df.index = range(len(t_matches_df["hPenalties"]))
t_matches_df.head()

,hTeam,hGoals,hPenalties,aTeam,aGoals,aPenalties,Match Type,Match Date
0,Italy,1,3,England,1,2,Final,11.07.21
1,Spain,1,2,Italy,1,4,Semi-finals,06.07.21
2,England,2,<NA>,Denmark,1,<NA>,Semi-finals,07.07.21
3,Switzerland,1,1,Spain,1,3,Quarter-finals,02.07.21
4,Belgium,1,<NA>,Italy,2,<NA>,Quarter-finals,02.07.21


### Pulling it All Together

Below will be re-iterated code, but formatted in a fashion that it iterates through multiple pages to coalcese data from each Euro's 1976 - 2020 including the Qualifiers if that is practical

In [ ]:
import requests 
import pandas as pd
import io
from bs4 import BeautifulSoup

t_url = "https://terrikon.com/en/euro-2020"
t_page = requests.get(t_url)
t_html = t_page.text
t_soup = BeautifulSoup(io.StringIO(t_html))

t_links = t_soup.find("div", class_="news")
t_links = t_links.find("dl").select("a")
years = [link.get("href") for link in t_links]
url = "https://terrikon.com"

all_matches = []

def collect_matches(): 
    for year in years:
        cur_url = f"{url}{year}"
        cur_page = requests.get(cur_url)
        cur_html = cur_page.text
        cur_soup = BeautifulSoup(io.StringIO(cur_html))
    
        cur_tables = cur_soup.find("div", class_="maincol")
        cur_headers = cur_tables.find_all("h2")
        year_name = year.split("/")[2].split("-")[1]
        if len(year.split("/")) > 3: 
            year_name = year_name + ", Qualifiers"
        cur_comps = ["".join(header.text.split(f"Euro-{year_name}."))[1:] for header in cur_headers]
    
        cur_sections = []
        for cnt, header in enumerate(cur_headers):
            tracker = header.next_sibling
            cur_html = ""
            next_header = False
            try:
                next_header = cur_headers[cnt +1]
            except:
                while tracker: 
                    cur_html = cur_html + str(tracker)
                    tracker = tracker.next_sibling
                if cur_html:
                    cur_sections.append(BeautifulSoup(cur_html).div)
                continue
            while next_header and tracker != next_header and tracker:
                    cur_html = cur_html + str(tracker)
                    tracker = tracker.next_sibling
            if cur_html:
                    cur_sections.append(BeautifulSoup(cur_html).div)
    
        for i, section in enumerate(cur_sections): 
            s_tester = cur_sections[i]
            s_table = s_tester.select("table.gameresult")[0].prettify()
            s_table = pd.read_html(io.StringIO(s_table))[0]
            s_table = s_table.drop(columns=[0])
            
            s_filler = [pd.NA for row in s_table.iterrows()]
            mType = cur_comps[i]
            cType = f"Euros {year_name}"
            if "Group" in mType:
                mType = "Group Stage"
            s_matchtype = [mType for row in s_table.iterrows()]
            s_competiton = [cType for row in s_table.iterrows()] 
            
            s_table = s_table.rename(columns={1:"hTeam", 2:"hGoals", 3:"aTeam", 4:"aGoals", 5:"mDate"})
            s_table.insert(2, "hPenalties", s_filler, True)
            s_table.insert(3, "hResult", s_filler, True)
            s_table.insert(6, "aPenalties", s_filler, True)
            s_table.insert(7, "aResult", s_filler, True)
            s_table.insert(8, "mType", s_matchtype, True)
            s_table.insert(10, "comp", s_competiton, True)
            
            for index, row in s_table.iterrows(): 
                scoring = row["hGoals"]
                if "+" in scoring or "-" in scoring: # means a match wasn't played
                    s_table.drop(index, inplace=True)
                    continue
                goals = scoring.split(" ")[0].split(":")
                hG = goals[0]
                hP = pd.NA
                hR = pd.NA
                aG = goals[1]
                aP = pd.NA
                aR = pd.NA
    
                s_table.loc[index, "hGoals"] = int(hG)
                s_table.loc[index, "aGoals"] = int(aG)
                
                if "(" in scoring: # penalties for match
                    penalties = scoring.split(" ")[-1].split(":")
                    hP = penalties[0][1:]
                    aP = penalties[1][:-1]
                    s_table.loc[index, "hPenalties"] = int(hP)
                    s_table.loc[index, "aPenalties"] = int(aP)
    
                    if hP > aP:
                        hR = "W"
                        aR = "L"
                    else: 
                        hR = "L"
                        aR = "W"
    
                    s_table.loc[index, "hResult"] = hR
                    s_table.loc[index, "aResult"] = aR
                    
                    continue 
            
                if hG > aG:
                    hR = "W"
                    aR = "L"
                elif hG < aG:
                    hR = "L" 
                    aR = "W"
                else:
                    hR = "D"
                    aR = "D"
    
                s_table.loc[index, "hResult"] = hR
                s_table.loc[index, "aResult"] = aR
            
            s_table[['hGoals', 'aGoals']] = s_table[['hGoals', 'aGoals']].astype('int')
            #s_table.dtypes
            all_matches.append(s_table)

collect_matches()
matches_df = pd.concat(all_matches)
matches_df.index = range(len(matches_df["hTeam"]))
matches_df.head()

In [ ]:
matches_df

In [ ]:
matches_df.to_csv("euros.csv")